<a href="https://colab.research.google.com/github/hadi-ansari/TFX/blob/main/Cat_%26_Dog_recognition_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple TFX Pipeline Tutorial using cats & dogs dataset

***A pipeline to train a model for recognizing cats and dogs.***

In this notebook-based tutorial, we will create and run a TFX pipeline
for a simple classification model.
The pipeline will consist of three essential TFX components: ExampleGen,
Trainer and Pusher. The pipeline includes the most minimal ML workflow like
importing data, training a model and exporting the trained model.

Please see
[Understanding TFX Pipelines](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)
to learn more about various concepts in TFX.

## Set Up
We first need to install the TFX Python package and download
the dataset which we will use for our model.

### Upgrade Pip

In [1]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [2]:
!pip install -U tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

### Uninstall shapely

TODO(b/263441833) This is a temporal solution to avoid an
ImportError. Ultimately, it should be handled by supporting a
recent version of Bigquery, instead of uninstalling other extra
dependencies.

In [3]:
!pip uninstall shapely -y

Found existing installation: shapely 2.0.1
Uninstalling shapely-2.0.1:
  Successfully uninstalled shapely-2.0.1


### Did you restart the runtime?

If you are using Google Colab, the first time that you run
the cell above, you must restart the runtime by clicking
above "RESTART RUNTIME" button or using "Runtime > Restart
runtime ..." menu. This is because of the way that Colab
loads packages.

# Necessary imports

We also check the TensorFlow and TFX versions.

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow_datasets as tfds
import tensorflow as tf
from absl import logging

logging.set_verbosity(logging.INFO)  # Set default logging level.

print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.11.1
TFX version: 1.12.0


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ismailsafwat1","key":"7b0531deedec78ff800fd18b88fd120b"}'}

Because TFX ExampleGen reads inputs from a directory, we need to create a
directory and copy dataset to it.

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json'
!kaggle datasets download -d salader/dogs-vs-cats
!unzip dogs-vs-cats.zip
!rm -rf test train dogs-vs-cats.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/dogs/dog.4419.jpg  
  inflating: train/dogs/dog.442.jpg  
  inflating: train/dogs/dog.4420.jpg  
  inflating: train/dogs/dog.4421.jpg  
  inflating: train/dogs/dog.4422.jpg  
  inflating: train/dogs/dog.4424.jpg  
  inflating: train/dogs/dog.4425.jpg  
  inflating: train/dogs/dog.4426.jpg  
  inflating: train/dogs/dog.4427.jpg  
  inflating: train/dogs/dog.4431.jpg  
  inflating: train/dogs/dog.4433.jpg  
  inflating: train/dogs/dog.4436.jpg  
  inflating: train/dogs/dog.4438.jpg  
  inflating: train/dogs/dog.4439.jpg  
  inflating: train/dogs/dog.444.jpg  
  inflating: train/dogs/dog.4440.jpg  
  inflating: train/dogs/dog.4441.jpg  
  inflating: train/dogs/dog.4442.jpg  
  inflating: train/dogs/dog.4443.jpg  
  inflating: train/dogs/dog.4444.jpg  
  inflating: train/dogs/dog.4445.jpg  
  inflating: train/dogs/dog.4446.jpg  
  inflating: train/dogs/dog.445.jpg  
  inflating: train/dogs/dog.4450.jpg  
  inflating: tra

# Reduce size of the dataset (optional)
We reduce size of the dataset by removing the images from both train and test folders. Train images decreas from 10000 images to 50 and test images decreas from 2500 to 10 images to make the process faster (OBS: just for testing).

In [4]:
!ls -1 dogs_vs_cats/train/dogs/* | tail -n +501 | xargs rm 
!ls -1 dogs_vs_cats/train/cats/* | tail -n +501 | xargs rm

!ls -1 dogs_vs_cats/test/dogs/* | tail -n +151 | xargs rm 
!ls -1 dogs_vs_cats/test/cats/* | tail -n +151 | xargs rm 

# Set up variables

There are some variables used to define a pipeline. You can customize these
variables as you want. By default all output from the pipeline will be
generated under the current directory.

In [5]:
_trainer_module_file = 'dogs_vs_cats_trainer.py'
schema_path = 'schema' # create this directory and copy your generated schema here in order to use the Importer.

data_root ='dogs_vs_cats'
train_dir = os.path.join(data_root, 'train')
validation_dir = os.path.join(data_root, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

IMG_HEIGHT = 150
IMG_WIDTH = 150
batch_size = 128
PIPELINE_NAME = 'dogs_vs_cats_pipeline'
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'cat&dog_md.db')


print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)
print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cat images: 500
total training dog images: 500
total validation cat images: 150
total validation dog images: 150
--
Total training images: 1000
Total validation images: 300


# Conversion of image dataset to TFRecords

We followed the instructions according [this](https://ai.plainenglish.io/a-quick-and-simple-guide-to-tfrecord-c421337a6562).


In [6]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import random
import cv2
import matplotlib.pyplot as plt
from io import BytesIO

# Setup the train and test imgage directories
train_dir = os.path.join(data_root, "train")
test_dir = os.path.join(data_root, "test")

# setup train and test TFRecord file
train_tfrecord='train_data.tfrecords'
test_tfrecord = 'test_data.tfrecords'

# Define the name of folders of each class
# We only have two classes in this case.
folders=['dogs', 'cats']

# List all train and test image path
train_image_path=[]
test_image_path=[]

for i in range(len(folders)):
    for file in os.listdir(os.path.join(train_dir, folders[i])):
        train_image_path.append(os.path.join(train_dir, folders[i], file))
    for file in os.listdir(os.path.join(test_dir, folders[i])):
        test_image_path.append( os.path.join(test_dir, folders[i], file))


print("Number of train images found: ", len(train_image_path))
print("Number of test images found: ", len(test_image_path))

# Shuffle the image paths for better accuracy and precision
random.seed(0)
random.shuffle(train_image_path)
random.shuffle(test_image_path)

# create train and test lables for shuffled image paths
# 0 for cat and 1 for dog
train_labels=[]
test_labels=[]
for i in range(len(train_image_path)):
    if os.path.basename(train_image_path[i])[:3]=='cat':
        train_labels.append(0)
    else:
        train_labels.append(1)

for i in range(len(test_image_path)):
    if os.path.basename(test_image_path[i])[:3]=='cat':
        test_labels.append(0)
    else:
        test_labels.append(1)

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""    
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_example(image_string, label):
    ## Create a dictionary with features for images and their target labels
    image_shape = tf.io.decode_jpeg(image_string).shape

    feature = {
      # 'height': _int64_feature(image_shape[0]),
      # 'width': _int64_feature(image_shape[1]),
      # 'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
    }
    #  Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    #serializes the message and returns it as a string. Note that the bytes are binary
    return example_proto.SerializeToString()

def write_TFRecord(image_path, label):
    # Load the image and resize it
    image = Image.open(image_path)
    image = image.resize([IMG_HEIGHT, IMG_WIDTH])

    # Convert the resized image to bytes
    with BytesIO() as output:
        image.save(output, format='JPEG')
        image_bytes = output.getvalue()

    # Serialize the example
    example = serialize_example(image_bytes, label)
    return example


#Write Train TFRecord file
with tf.io.TFRecordWriter(train_tfrecord) as writer:
    for image_path, label in zip(train_image_path, train_labels):
        writer.write(write_TFRecord(image_path, int(label)))
#Write Test TFRecord file
with tf.io.TFRecordWriter(test_tfrecord) as writer:
    for image_path, label in zip(test_image_path, test_labels):
         writer.write(write_TFRecord(image_path, int(label)))

Number of train images found:  1000
Number of test images found:  300


Restructures the files and creates a directory named "dataset".

In [7]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/test
!mv train_data.tfrecords dataset/train
!mv test_data.tfrecords dataset/test

# Anomaly test
This code is to test anomaly and ExampleValidator component. Do **not** run this code if you do not want to add a invalid data into the dataset.

In [ ]:
with tf.io.TFRecordWriter(train_tfrecord) as writer:
    for image_path, label in zip(train_image_path, train_labels):
      writer.write(write_TFRecord(image_path, 3))
      break

# Trainer and Transform code


In [15]:
%%writefile {_trainer_module_file}

import tensorflow as tf
from tfx import v1 as tfx
import matplotlib.pyplot as plt
import tensorflow_transform as tft
from tfx_bsl.tfxio import dataset_options
import os
import gzip
from typing import List
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform.tf_metadata import schema_utils

_IMAGE_KEY = 'image_raw'
_LABEL_KEY = 'label'
IMG_HEIGHT = 150
IMG_WIDTH = 150
batch_size = 128

_TRAIN_DATA_SIZE = 1000
_TEST_DATA_SIZE = 300


def _transformed_name(key):
  return key + '_xf'

# TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}

  image_features = tf.map_fn(
      lambda x: tf.io.decode_jpeg(x[0], channels=3),
      inputs[_IMAGE_KEY],
      dtype=tf.uint8)
  
  image_features = tf.image.resize(image_features, [IMG_HEIGHT, IMG_WIDTH])
  # Normalize the image features
  # image_features = image_features / 255.0

  image_features = tf.keras.applications.mobilenet.preprocess_input(image_features)

  outputs[_transformed_name(_IMAGE_KEY)] = image_features
  outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]
  
  return outputs


# Define a function to parse the tfrecord dataset
def _parse_function(features, label):
   image = features[_transformed_name(_IMAGE_KEY)]
   return image, label

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 128) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """  
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema).repeat()
  
  return dataset


def _get_serve_image_fn(model):
  """Returns a function that feeds the input tensor into the model."""

  @tf.function
  def serve_image_fn(image_tensor):
    """Returns the output to be used in the serving signature.
    Args:
      image_tensor: A tensor represeting input image. The image should have 3
        channels.
    Returns:
      The model's predicton on input image tensor
    """
    return model(image_tensor)

  return serve_image_fn


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
    
    print("\n\n\n Trainer component is running ... \n\n\n")

    epochs = 1
    #Create the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), name=_transformed_name(_IMAGE_KEY)),
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(),

        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(),

        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
 
    # Print a summary of the model
    model.summary()

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
 
    train_ds = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=batch_size)
    #print(train_ds)
    train_ds = train_ds.map(_parse_function)

    eval_ds = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=batch_size)
    
    eval_ds = eval_ds.map(_parse_function)

    steps_per_epoch = int(_TRAIN_DATA_SIZE / batch_size)
    validation_steps = int(_TEST_DATA_SIZE / batch_size)

    # Train our model
    history = model.fit(train_ds,
                        epochs=epochs,
                        validation_data=eval_ds, 
                        steps_per_epoch=steps_per_epoch, 
                        validation_steps=validation_steps)




    # acc = history.history['accuracy']
    # val_acc = history.history['val_accuracy']

    # loss = history.history['loss']
    # val_loss = history.history['val_loss']

    # epochs_range = range(epochs)
    # plt.figure(figsize=(8, 8))
    # plt.subplot(1, 2, 1)
    # plt.plot(epochs_range, acc, label='Training Accuracy')
    # plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    # plt.legend(loc='lower right')
    # plt.title('Training and Validation Accuracy')

    # plt.subplot(1, 2, 2)
    # plt.plot(epochs_range, loss, label='Training Loss')
    # plt.plot(epochs_range, val_loss, label='Validation Loss')
    # plt.legend(loc='upper right')
    # plt.title('Training and Validation Loss')
    # plt.show()


    # Prepare the TFLite model used for serving in MLKit
    signatures = {
        'serving_default':
            _get_serve_image_fn(model).get_concrete_function(
                tf.TensorSpec(
                    shape=[None, IMG_HEIGHT, IMG_WIDTH, 3],
                    dtype=tf.float32,
                    name=_transformed_name(_IMAGE_KEY)))
    }



    # The result of the training should be saved in `fn_args.serving_model_dir`
    # directory.
    model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting dogs_vs_cats_trainer.py


#Pipeline Definition
We define the ML pipeline here. The components of the pipeline are: ImportExampleGen for ingesting the images into the pipeline, StatisticsGen and SchemaGen/Importer, Transform, Trainer, Resolver, Evaluator and Pusher. 

For ingesting the image dataset into the pipeline, we got help from this github repository [github](https://github.com/tensorflow/tfx/blob/master/tfx/examples/cifar10/cifar10_pipeline_native_keras.py)

## Alternative 1
Run this code if you do **not** have a schema.

In [27]:
!rm -rf ./serving_model/

In [28]:
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.dsl.components.base import executor_spec
from tfx.components.example_gen.csv_example_gen import executor
from tfx.dsl.components.base.base_executor import BaseExecutor
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
import tensorflow_model_analysis as tfma
from tfx.types import Channel

def _create_pipeline(pipeline_name: str, pipeline_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str, accuracy_threshold: float = 0.55) -> tfx.dsl.Pipeline:
  print("Pipeline creation is running...\n\n\n")

  """Creates a three component penguin pipeline with TFX."""
 
  input_config = example_gen_pb2.Input(splits=[
      example_gen_pb2.Input.Split(name='train', pattern='train/*'),
      example_gen_pb2.Input.Split(name='eval', pattern='test/*')
  ])

  # Brings data into the pipeline.
  example_gen = tfx.components.ImportExampleGen(
      input_base='/content/dataset', input_config=input_config)

  # Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
  
  # In case you do have not generated schema, opt for this method.
  # Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
  
  #Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema = schema_gen.outputs['schema']
  )

  # Performs transformations and feature engineering in training and serving.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)
  
  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=transform.outputs['transformed_examples'],
      transform_graph=transform.outputs['transform_graph'],
      schema= schema_gen.outputs['schema']
      )
  
 # Get the latest blessed model for model validation.
  model_resolver = resolver.Resolver(
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(
          type=ModelBlessing)).with_id('latest_blessed_model_resolver')


  # Uses TFMA to compute evaluation statistics over features of a model and
  # perform quality validation of a candidate model (compare to a baseline).
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='label_xf')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-3})))
          ])
      ])

  # Uses TFMA to compute the evaluation statistics over features of a model.
  # We evaluate using the materialized examples that are output by Transform
  # because
  # 1. the decoding_png function currently performed within Transform are not
  # compatible with TFLite.
  # 2. MLKit requires deserialized (float32) tensor image inputs
  # Note that for deployment, the same logic that is performed within Transform
  # must be reproduced client-side.
  model_analyzer = tfx.components.Evaluator(
      examples=transform.outputs['transformed_examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)
  
  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=model_analyzer.outputs['blessing'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))
  
  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      statistics_gen,
      schema_gen,
      example_validator,
      transform,
      trainer,
      model_resolver, 
      model_analyzer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

# Alternative 2
Run this code if you already have a schema.

In [17]:
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.dsl.components.base import executor_spec
from tfx.components.example_gen.csv_example_gen import executor
from tfx.dsl.components.base.base_executor import BaseExecutor
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
import tensorflow_model_analysis as tfma
from tfx.types import Channel

from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types.standard_artifacts import Model, ModelBlessing

def _create_pipeline(pipeline_name: str, pipeline_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str, accuracy_threshold: float = 0.55) -> tfx.dsl.Pipeline:
  print("Pipeline creation is running...\n\n\n")

  """Creates a three component penguin pipeline with TFX."""
 
  input_config = example_gen_pb2.Input(splits=[
      example_gen_pb2.Input.Split(name='train', pattern='train/*'),
      example_gen_pb2.Input.Split(name='eval', pattern='test/*')
  ])
  # Brings data into the pipeline.
  example_gen = tfx.components.ImportExampleGen(
      input_base='/content/dataset', input_config=input_config)

  # Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
  
  # In case you have generated schema before, opt for this alternative.
  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri = schema_path,
      artifact_type = tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result']
  )

  # Performs transformations and feature engineering in training and serving.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      module_file=module_file)
  
  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=transform.outputs['transformed_examples'],
      transform_graph=transform.outputs['transform_graph'],
      schema=schema_importer.outputs['result']
      )
  
  # Get the latest blessed model for model validation.
  model_resolver = resolver.Resolver(
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(
          type=ModelBlessing)).with_id('latest_blessed_model_resolver')


  # Uses TFMA to compute evaluation statistics over features of a model and
  # perform quality validation of a candidate model (compare to a baseline).
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='label_xf')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-3})))
          ])
      ])

  # Uses TFMA to compute the evaluation statistics over features of a model.
  # We evaluate using the materialized examples that are output by Transform
  # because
  # 1. the decoding_png function currently performed within Transform are not
  # compatible with TFLite.
  # 2. MLKit requires deserialized (float32) tensor image inputs
  # Note that for deployment, the same logic that is performed within Transform
  # must be reproduced client-side.
  model_analyzer = tfx.components.Evaluator(
      examples=transform.outputs['transformed_examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)
  
  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=model_analyzer.outputs['blessing'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))
  
  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      #statistics_gen,
      schema_importer,
      #example_validator,
      transform,
      trainer,
      model_resolver,
      model_analyzer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

# Run the pipeline

TFX supports multiple orchestrators to run pipelines.
In this tutorial we will use `LocalDagRunner` which is included in the TFX
Python package and runs pipelines on local environment.
We often call TFX pipelines "DAGs" which stands for directed acyclic graph.

`LocalDagRunner` provides fast iterations for development and debugging.
TFX also supports other orchestrators including Kubeflow Pipelines and Apache
Airflow which are suitable for production use cases.

See
[TFX on Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)
or
[TFX Airflow Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop)
to learn more about other orchestration systems.

Now we create a `LocalDagRunner` and pass a `Pipeline` object created from the
function we already defined.

The pipeline runs directly and you can see logs for the progress of the pipeline including ML model training.

In [29]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/dogs_vs_cats_trainer.py' (including modules: ['dogs_vs_cats_trainer']).
INFO:absl:User module package has hash fingerprint version 1740379d8c1a360c5e255524f7978080a2996d90de64fd6077352c5e3f7fcda4.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpkdd4aqiu/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp2w0jnuqm', '--dist-dir', '/tmp/tmpy2oy7ew1']


Pipeline creation is running...





Streaming output truncated to the last 5000 lines.
  }
}
, pipeline_info=id: "dogs_vs_cats_pipeline"
, pipeline_run_id='2023-04-20T15:26:49.634243')
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to pipelines/dogs_vs_cats_pipeline/ExampleValidator/anomalies/69/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to pipelines/dogs_vs_cats_pipeline/ExampleValidator/anomalies/69/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 69 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'anomalies': [Artifact(artifact: uri: "pipelines/dogs_vs_cats_pipeline/ExampleValidator/anomalies/69"
, artifact_type: name: "ExampleAnomalies"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names




 Trainer component is running ... 



Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_raw_xf (Conv2D)       (None, 150, 150, 32)      896       
                                                                 
 conv2d_48 (Conv2D)          (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_49 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 conv2d_50 (Conv2D)          (None, 75, 75, 64)        36928     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 37, 37, 64)       0         
 g2D)        

INFO:absl:Feature image_raw_xf has a shape dim {
  size: 150
}
dim {
  size: 150
}
dim {
  size: 3
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_raw_xf has a shape dim {
  size: 150
}
dim {
  size: 150
}
dim {
  size: 3
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_raw_xf has a shape dim {
  size: 150
}
dim {
  size: 150
}
dim {
  size: 3
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_raw_xf has a shape dim {
  size: 150
}
dim {
  size: 150
}
dim {
  size: 3
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.


7/7 [==============================] - 11s 688ms/step - loss: 0.7467 - accuracy: 0.4688 - val_loss: 0.6927 - val_accuracy: 0.4961


INFO:absl:Training complete. Model written to pipelines/dogs_vs_cats_pipeline/Trainer/model/71/Format-Serving. ModelRun written to pipelines/dogs_vs_cats_pipeline/Trainer/model_run/71
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 71 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/dogs_vs_cats_pipeline/Trainer/model/71"
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "pipelines/dogs_vs_cats_pipeline/Trainer/model_run/71"
, artifact_type: name: "ModelRun"
)]}) for execution 71
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Trainer is finished.
INFO:absl:Component Evaluator is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.evaluator.component.Evaluator"
    base_type: EVALUATE
  }
  id: "Evaluator"
}
contexts {
  contexts {
    type {
    

You should see "INFO:absl:Component Pusher is finished." at the end of the
logs if the pipeline finished successfully. Because `Pusher` component is the
last component of the pipeline.

The pusher component pushes the trained model to the `SERVING_MODEL_DIR` which
is the `serving_model/penguin-simple` directory if you did not change the
variables in the previous steps. You can see the result from the file browser
in the left-side panel in Colab, or using the following command:

# Review outputs of the pipeline

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

In [ ]:
#@title
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

In [ ]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'depth',INT,required,,-
'height',INT,required,,-
'image_raw',BYTES,required,,-
'label',INT,required,,-
'width',INT,required,,-


In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

In [ ]:
visualize_artifacts(anomalies_artifacts)

In [ ]:
eval_ds = tf.keras.utils.image_dataset_from_directory(
    "./dogs_vs_cats/test",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
)


model = tf.keras.models.load_model('/content/serving_model/dogs_vs_cats_pipeline/1681909098')
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



predictions = probability_model.predict(eval_ds)

In [ ]:
# Use the trained model to predict on the eval_ds dataset
predictions = model.predict(eval_ds)

# Print the predictions for the first 10 images in the dataset
for image, prediction in zip(eval_ds, predictions):
    predicted_label = "dog" if prediction > 0.5 else "cat"
    true_label = "dog" if image[1].numpy()[0] == 1 else "cat"
    print("Predicted label: ", predicted_label)
    print("True label: ", true_label)